# A2 — Regressão de Deriva da IMU (Fácil)

Estime a magnitude do bias (deriva) do giroscópio a partir de janelas curtas de IMU. Prefira **feature map quântico + ridge** ou **regressor VQC**.

### Contexto
Uma IMU (Unidade de Medição Inercial) fornece sensoriamento de movimento em 6 eixos: acelerômetro 3 eixos $(a_x,a_y,a_z)$ e giroscópio 3 eixos $(g_x,g_y,g_z)$. Giroscópios medem velocidade angular, mas sensores reais sofrem com um offset lentamente variável (bias) que acumula erro quando integrado.

#### O que é bias/deriva do giroscópio?
Modelo simples: $g_{\text{meas}}(t) = g_{\text{true}}(t) + b_g + \eta(t)$, onde $b_g$ é um bias quase constante e $\eta$ é ruído. Mesmo um bias pequeno (ex.: $0.01\text{–}0.03$ rad/s) causa deriva perceptível de rumo ao longo de minutos.

#### Por que estimar o bias?
- Estabilizar estimativas de atitude/rumo em odometria por rodas/pernas e VIO.
- Melhorar estimação de estado em períodos de baixa excitação (parado/pairando).
- Reduzir frequência de recalibração e permitir compensação online em controladores.

#### Dataset e alvo
Cada amostra é uma janela curta $(W\!\times\!6)$ da IMU. O alvo é a magnitude escalar do bias do giroscópio $\lVert b_g \rVert$ sintetizada por [`common.data_utils.imu_drift_dataset`](common/data_utils.py). O comprimento da janela ($W=32$) mantém custo computacional e contagem de qubits gerenciáveis.

#### Por que métodos quânticos aqui?
Após comprimir janelas em estatísticas robustas, a regressão pode se beneficiar de feature maps não lineares expressivos. Feature maps quânticos ou pequenos VQCs atuam como enriquecedores de kernel sob orçamento reduzido de qubits, potencialmente melhorando generalização em regime de poucos dados.

## Especificação de Entrada e Saída
**Entrada:** `X_train (N×W×6)`, `y_train (N,)`, `X_test (M×W×6)` com $W=32$ nos dados fornecidos  
**Saída:** `y_pred (M,)` magnitude do bias em rad/s  
**Tipo retornado:** array float (ex.: `np.float64`)

Faixa esperada real do bias ≈ $0.01$–$0.03$ rad/s; previsões fora de $[0.0, 0.05]$ são possivelmente errôneas.  
Orientações de recursos: qubits ≤ 10, passos do otimizador ≤ 150; evitar loops Python pesados por amostra.

### Dicas de engenharia de características
- O bias aparece como offset quase constante nos canais do giroscópio (índices 3–5).
- A média temporal dos eixos do giroscópio é forte estimador; variância ajuda a separar ruído vs. deriva.
- Extraia (média, mediana, desvio padrão, MAD) por eixo do giroscópio; agregue estatísticas do acelerômetro para reduzir correlação com movimento.
- Abordagem quântica: condense estatísticas em embedding de baixa dimensão e aplique um feature map quântico antes de regressão linear.

### Notas de pré-processamento
- Padronize (zero média, desvio padrão 1) as features antes da regressão para estabilizar a ridge.
- Evite vazamento de estatísticas do teste: ajuste scalers só em treino.
- Mantenha qubits modestos (≤10) reduzindo para vetor compacto antes da codificação quântica.

## Setup

In [ ]:
import numpy as np
from common import data_utils as du
from common import baselines as bl
from common import quantum_utils as qu
np.random.seed(1337)


## Baseline (referência)

_Orientação de baseline:_ em seeds variados com [`common.data_utils.imu_drift_dataset`](common/data_utils.py) o ridge clássico tipicamente obtém MAE $0.012\text{–}0.018$ rad/s; o limite público em $0.020$ deixa margem para melhorias quânticas. Mantenha previsões dentro do intervalo plausível $[0.0,0.05]`.

In [ ]:
Xtr, ytr = du.imu_drift_dataset(n=320, w=32, seed=0)
Xte, yte = du.imu_drift_dataset(n=80,  w=32, seed=5)
yp = bl.baseline_imu_reg(Xtr, ytr, Xte)
mae = np.mean(np.abs(yp - yte))
print(f"Baseline MAE: {mae:.4f} rad/s")


## Sua tarefa: implementar `solve(...)`

In [ ]:
def solve(Xtr, ytr, Xte):
    # Substitua por regressor com enriquecimento quântico (feature map + cabeça ridge ou VQC).
    # Use common.quantum_utils.feature_map em features comprimidas.
    return bl.baseline_imu_reg(Xtr, ytr, Xte)

## Testes públicos (rápidos)

In [ ]:
Xtr, ytr = du.imu_drift_dataset(n=320, w=32, seed=10)
Xte, yte = du.imu_drift_dataset(n=80,  w=32, seed=11)
yp = solve(Xtr, ytr, Xte)
mae = np.mean(np.abs(yp - yte))
print("Public MAE:", mae)
assert yp.shape == yte.shape
assert mae <= 0.020, "MAE público acima do limiar"
print("OK")


> Testes adicionais serão executados pelos organizadores com seeds/tamanhos diferentes.